In [4]:
import numpy as np
from itertools import combinations
from sage.all import QQ, NumberField, PolynomialRing

In [5]:
def find_feasible_vertices(A, b):
    """Find vertices of polytope defined by A @ x ≤ b"""
    num_vars = A.shape[1]
    vertices = []
    
    for indices in combinations(range(len(A)), num_vars):
        A_sub = A[list(indices)]
        b_sub = b[list(indices)]
        
        try:
            vertex = np.linalg.solve(A_sub, b_sub)
            if np.all(A @ vertex <= b + 1e-6):
                vertices.append(vertex)
        except np.linalg.LinAlgError:
            continue
    
    return np.unique(np.round(vertices, decimals=6), axis=0) if vertices else np.array([])




def twoParts(alpha, embeddings):
    """FIELD INPUT Calculation"""
    K = alpha.parent().fraction_field()  
    
    if not K.is_totally_real():
        raise ValueError("Field must be totally real")
    
    O = K.maximal_order()
    basis = O.basis()
    """FIELD INPUT Calculation Over"""

    """BOUNDS CALCULATION"""
    embedding_data = []
    for sig in embeddings:
        a_img = sig(basis[0]) if len(basis) > 0 else 0
        b_img = sig(basis[1]) if len(basis) > 1 else 0
        alpha_img = sig(alpha)
        embedding_data.append((a_img, b_img, alpha_img))
 
    A_ineq = []
    b_ineq = []
    epsilon = 1e-6
    
    for (a, b, Alpha) in embedding_data:
        A_ineq.append([float(a), float(b)])
        b_ineq.append(float(Alpha) - epsilon)
        A_ineq.append([-float(a), -float(b)])
        b_ineq.append(-epsilon)

    A_ineq = np.array(A_ineq)
    b_ineq = np.array(b_ineq)
    
    vertices = find_feasible_vertices(A_ineq, b_ineq)
    
    if not vertices.size:
        return 0, []
    
    bound = []
    
    for i in range(2):
        a = []
        a.append(np.floor(np.min(vertices[:, i])))
        a.append(np.ceil(np.max(vertices[:, i])))
        bound.append(a)
  
    k1_min, k1_max = bound[0][0], bound[0][1]
    k2_min, k2_max = bound[1][0], bound[1][1]

    """BOUNDS CALCULATION OVER"""

    """FINDING PARTITIONS"""
    partitions = []
    for k1 in range(int(k1_min), int(k1_max) + 1):
        for k2 in range(int(k2_min), int(k2_max) + 1):
            beta = k1 * basis[0] + k2 * basis[1] 
         
            Y = alpha - beta
            
            if beta == 0 or Y == 0:
                continue
            
            if beta.is_totally_positive() and Y.is_totally_positive():
                partitions.append((beta, Y))
    
    unique_partitions = []
    seen = set()
    for pair in partitions:
        if pair not in seen and pair[::-1] not in seen:
            seen.add(pair)
            unique_partitions.append(pair)
    
    return unique_partitions

def allParts(alpha, embeddings, memo=None):
    """
    Recursively find all partitions of alpha into sums of totally positive elements.
    Stops recursion when no further partitions are possible.
    Returns the number of unique partitions.
    """
    if memo is None:
        memo = {}                               
    
    alpha_key = tuple(alpha.list())
    if alpha_key in memo:
        return memo[alpha_key]
    
    
    partitions = twoParts(alpha, embeddings)

    all_partitions = []
    
    for beta, gamma in partitions:
        all_partitions.append([beta, gamma])
    
        beta_partitions = allParts(beta, embeddings, memo)
        for p in beta_partitions:
            all_partitions.append(p + [gamma])
        
        gamma_partitions = allParts(gamma, embeddings, memo)
        for p in gamma_partitions:
            all_partitions.append([beta] + p)

    unique_partitions = []
    seen = set()
    for partition in all_partitions:
        sorted_part = tuple(sorted(tuple(x.list()) for x in partition))
        
        if sorted_part not in seen:
            seen.add(sorted_part)
            unique_partitions.append(partition)
    
    memo[alpha_key] = unique_partitions

    return unique_partitions

def Partitions(alpha, embeddings):
    memo={}
    if alpha.is_totally_positive():
        partitions = allParts(alpha,embeddings,memo)
        return len(partitions)+1,memo
    else:
        return "The number given is not totally positive"

def memolength(memo):
    """CALCULATES LENGTH OF MEMO"""
    lmemo={}
    for key in memo:
        lmemo[key]=len(memo[key])
    return(lmemo)



In [6]:
R.<x> = QQ[]
K.<x> = NumberField(x^2-2)
O = K.ring_of_integers()
embeddings = K.embeddings(QQbar)  

alpha=14-2*x
alpha1 = O.coordinates(alpha)[0]*O.basis()[0] + O.coordinates(alpha)[1]*O.basis()[1]
l,fmemo=Partitions(alpha1, embeddings)

print(l)

1415


The below codes are to write CSV files etc.

In [23]:
import csv

def Partition_Function(D, a, b):
    R.<x> = QQ[]
    K.<x> = NumberField(x^2 - D)
    O = K.ring_of_integers()
    embeddings = K.embeddings(QQbar) 
    
    if D % 4 == 1:
        alpha = a + b * O.basis()[0]
    else:
        alpha = a + b * x
    
    l, fmemo = Partitions(alpha, embeddings)
    
    # Writing to CSV file
    with open("partition_results5.csv", mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([a, b, l])


 


In [24]:
for j in range(16):
    for i in range(16):
        try:
            Partition_Function(5,j,i)
        except ValueError:
            continue